In [1]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from matplotlib import rcParams
from matplotlib import animation
from IPython.display import HTML
import matplotlib.cm as cm 
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [2]:
def initial_state(nx,ny):
    """ Creates a lattice with random spin orientations
   
    Parameters:
    ----------
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
   
    Returns:
    -------
    r: 2D array of spins
    """
       
    # Uniform random spins between 0 and 1  
    r = np.random.randint(0,2,(nx,ny))
    
    # Where r=0, we assign a negative spin
    r[r==0] =-1

    return r

In [3]:
def energy_at_site(sp,alpha,sigma,sigma1,ix,iy):
    """ Computes the contribution to the energy for a given spin
    at location ix,iy for a lattice with periodic boundary conditions
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    sigma   : int
        spin at site ix,iy
    ix   : int
        location in x
    iy   : int
        location in y
    
   
    Returns:
    -------
    energy: energy for the given configuration
    """
    
    #bordure composé de spin up uniquement
    if ix==0 and iy!=0 and iy!=ny:
        energy=-alpha*sigma1*(sp[(ix+1),iy]+sp[ix,(iy-1)]+sp[ix,(iy+1)])
    
    elif ix==nx and iy!=0 and iy!=ny:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[ix,(iy-1)]+sp[ix,(iy+1)])
    
    elif iy==0 and ix!=0 and ix!=nx:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[(ix+1),iy]+sp[ix,(iy+1)])
    
    elif iy==ny and ix!=0 and ix!=nx:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[(ix+1),iy]+sp[ix,(iy-1)])
    #coins    
    elif ix==0 and iy==0:
        energy=-alpha*sigma1*(sp[(ix+1),iy]+sp[ix,(iy+1)])
    
    elif ix==nx and iy==0:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[ix,(iy+1)])
    
    elif iy==ny and ix==0:
        energy=-alpha*sigma1*(sp[(ix+1),iy]+sp[ix,(iy-1)])
    
    elif iy==ny and ix==nx:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[ix,(iy-1)])
    #carré intérieur
    else: 
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[(ix+1),iy]+sp[ix,(iy-1)]+sp[ix,(iy+1)])
        
    return energy

In [4]:
def metropolis_at_site(sp,sigma1,alpha,ix,iy):
    """ Flips a dipole at site ix, iy when probability condition is met 
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    ix   : int
        location in x
    iy   : int
        location in y
    """
    sigma=sp[ix,iy]
    energy_before_flip = energy_at_site(sp,alpha,sigma,sigma1,ix,iy)
    sigma = -sigma
    energy_if_site_flipped = energy_at_site(sp,alpha,sigma,sigma1,ix,iy)
    
    # Flip the site with Metropolis probability
    # Condition is always satisifed if dE < 0
    if (np.random.random_sample()<np.exp(-(energy_if_site_flipped \
                                               -energy_before_flip))):
        sp[ix,iy]=-sp[ix,iy]

In [18]:
def ising_model_metropolis(sp,sigma1, NMC, nx, ny, alpha):
    """ Creates a sequence of states for the Ising model using
    the Metropolis algorithm
   
    Parameters:
    ----------
    sp   : initial lattice state
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
    NMC  : int
        Number of states to create
    alpha  : real
        coupling constant J/(kb*T)
    Returns:
    -------
    states: sequence of states
    """
    states = np.empty([NMC+1,nx-1,ny-1])
    states[0] = sp.copy()
    
    for i in range(1,NMC+1):
        for j in range(0,(nx-1)*(ny-1)):
            ix=np.random.randint(0,nx-1)
            iy=np.random.randint(0,ny-1)
            metropolis_at_site(sp,sigma1,alpha,ix,iy)
        states[i]=sp.copy()
    return states

In [19]:
sigma1=1
sigma=1
nx=100
ny=100
ix=5
iy=5
sp=np.ones([ny,nx])
NMC=100
alpha=0.7

In [20]:
energy_at_site(sp,alpha,sigma,sigma1,ix,iy)
ising_model_metropolis(sp,sigma1, NMC, nx, ny, alpha)

ValueError: could not broadcast input array from shape (100,100) into shape (99,99)